In [1]:
#Relational Database

import json
import mysql.connector
from datetime import datetime
from datetime import timedelta

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="12345678",
  database= "twitter"
)

def main():
    lines = 0
    num_tweets = 0
    num_retweets = 0
    print_tweet = 0
    print_retweet = 0
    tweets = {}

    users = []

    # insert path and replace name of the file below as needed
    with open("corona-out-2", "r") as f1:
        print("database connection completed")
        mycursor = mydb.cursor()

        for line in f1:

            try:
                data = json.loads(line)
                lines = lines + 1
                val = (
                        data['user']['id'], data['user']['name'], data['user']['screen_name'], data['user']['location'],
                        data['user']['url'], data['user']['description'], data['user']['translator_type'],
                        data['user']['protected'],
                        data['user']['verified'], data['user']['followers_count'], data['user']['friends_count'],
                        data['user']['listed_count'], data['user']['favourites_count'], data['user']['statuses_count'], datetime.strptime(data['user']['created_at'], '%a %b %d %H:%M:%S %z %Y'))
                users.append(val)
                if (data['text'].startswith('RT')):
                    num_retweets += 1
                    if (print_retweet < 2):
                        print_retweet += 1
                else:
                    num_tweets += 1
                    if (print_tweet < 2):
                        print_tweet += 1

                if (data['id_str'] not in tweets):
                    tweets[data['id_str']] = data
            except:
                continue

    with open("corona-out-3", "r") as f1:
        for line in f1:

            try:
                data = json.loads(line)
                lines = lines + 1
                val = (
                        data['user']['id'], data['user']['name'], data['user']['screen_name'], data['user']['location'],
                        data['user']['url'], data['user']['description'], data['user']['translator_type'],
                        data['user']['protected'],
                        data['user']['verified'], data['user']['followers_count'], data['user']['friends_count'],
                        data['user']['listed_count'], data['user']['favourites_count'], data['user']['statuses_count'], datetime.strptime(data['user']['created_at'], '%a %b %d %H:%M:%S %z %Y'))
                users.append(val)
                if (data['text'].startswith('RT')):
                    num_retweets += 1
                    if (print_retweet < 2):
                        print_retweet += 1
                else:
                    num_tweets += 1
                    if (print_tweet < 2):
                        print_tweet += 1

                if (data['id_str'] not in tweets):
                    tweets[data['id_str']] = data
            except:
                continue

        print("Tweers",num_tweets, "retweets",num_retweets)
        userMap = {}
        for user in users:
            if user[0] in userMap:
                if userMap[user[0]][14] < user[14]:
                    userMap[user[0]] = user
            else:
                userMap[user[0]] = user

        print(len(userMap))
        for id in userMap:
            user = userMap[id]
            try:
                # print("Insertion started")
                sql = "INSERT INTO user (id, name, screen_name,location, url , description, translator_type ,protected,verified," \
                      "followers_count, friends_count,listed_count,favourites_count,statuses_count, created_at) VALUES (%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s, %s, %s)"


                val = (user[0], user[1], user[2], user[3], user[4], user[5], user[6], user[7], user[8], user[9], user[10], user[11]
                       , user[12], user[13], user[14])
                # print("val:", val)
                mycursor.execute(sql, val)
            except mysql.connector.Error as err:
                print(" 2   Something went wrong: {}".format(err))


        print("Commiting")
        mydb.commit()
    # print('num of lines=', lines, 'num of tweets=', num_tweets, 'num of retweets=', num_retweets)
    # print('num of unique tweets/retweets=', len(tweets.keys()))


# select id,name,screen_name,verified from user
# where name like ("j%") or screen_name like ("j%") group by 1,2,3,4;
#
# select location,id,name,screen_name,verified from user group by 1,2,3,4,5;
#
# select id,name,screen_name,verified from user where verified = 1;

def search_by_name(name):
    print("Opening database connection")
    mycursor = mydb.cursor()
    try:
        print("Select by name or screen name starting")
        sql = "SELECT id,name,screen_name,verified from USER where name like CONCAT(%s,'%') OR screen_name like CONCAT(%s,'%') group by 1,2,3,4"
        val = (name,name,)
        mycursor.execute(sql, val)
        results = mycursor.fetchall()
        print(results)
    except mysql.connector.Error as err:
        print("Something went wrong: {}".format(err))
    print("Search Commited")
    return results

def search_by_location(location):
    print("Opening database connection")
    mycursor = mydb.cursor()
    try:
        print("Select by location starting")
        sql = "SELECT location,id,name,screen_name,verified from USER where location like CONCAT(%s,'%') group by 1,2,3,4,5"
        val = (location,)
        mycursor.execute(sql, val)
        results = mycursor.fetchall()
        print(results)
    except mysql.connector.Error as err:
        print("Something went wrong: {}".format(err))
    print("Search Commited")
    return results

def search_by_verified(status):
    print("Opening database connection")
    mycursor = mydb.cursor()
    try:
        print("Select by location starting")
        sql = "SELECT id,name,screen_name,verified from USER where verified = %s"
        val = (1 if status else 0,)
        mycursor.execute(sql, val)
        results = mycursor.fetchall()
        print(results)
    except mysql.connector.Error as err:
        print("Something went wrong: {}".format(err))
    print("Search Commited")
    return results



In [ ]:
#Non Relational Database

import json
import pymongo

from pymongo import MongoClient
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client['twitter']
print("Database is created !!")
print(db)

collection = db["retweets"]
print("List of databases after creating new one")
print(client.list_database_names())
cursor = collection.find()




with open("./corona-out-2.txt", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)

            if (data['text'].startswith('RT')):
                data_retweet = {'text': data['text'], 'timestamp_ms': data['timestamp_ms'],
                                'created_at': data['created_at'], 'name': data['user']['name'],
                                'Followers_count': data['user']['followers_count'],
                                'Retweet_count': data['retweeted_status']['retweet_count']}
                collection.insert_one(data_retweet)
            else:
                continue
        except:
            continue

with open("./corona-out-2.txt", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            if not (data['text'].startswith('RT')):
                data_tweet = {'text': data['text'], 'timestamp_ms': data['timestamp_ms'],
                              'created_at': data['created_at'], 'name': data['user']['name'],
                              'Followers_count': data['user']['followers_count']}
                collection.insert_one(data_tweet)
            else:
                continue
        except:
            continue



def search_by_string(text, start_year_para, end_year_para):
    import datetime
    new_list = []
    new_list_with_timerange = []
    new_list_withtimerange_relevance = []
    for doc in db.my_tweet_project.find():
        if (text in doc['text']):
            new_list.append(doc)

    for doc in new_list:
        doc["timestamp_ms_ts"] = datetime.datetime.fromtimestamp((int(doc["timestamp_ms"])) / 1000).strftime(
            '%Y-%m-%d %H:%M:%S')
        doc["timestamp_ms_ts_year"] = datetime.datetime.fromtimestamp((int(doc["timestamp_ms"])) / 1000).strftime('%Y')

    # Time range search
    for doc in new_list:
        if (int(doc["timestamp_ms_ts_year"]) in range(start_year_para, end_year_para)):
            new_list_with_timerange.append(doc)

        # Notion of relevance to rank the search results
    new_list_with_timerange_relevance = sorted(new_list_with_timerange, key=lambda x: x['Followers_count'],
                                               reverse=True)
    return new_list_with_timerange_relevance


def search_by_hashtag(name, start_year_para, end_year_para):
    import datetime
    my_new_list = []
    my_new_list_with_timerange = []
    for doc in collection.find({'hashtags': name}, {"text": 1, "timestamp_ms": 1}):
        my_new_list.append(doc)

    for doc in my_new_list:
        doc["timestamp_ms_ts"] = datetime.datetime.fromtimestamp((int(doc["timestamp_ms"])) / 1000).strftime(
            '%Y-%m-%d %H:%M:%S')
        doc["timestamp_ms_ts_year"] = datetime.datetime.fromtimestamp((int(doc["timestamp_ms"])) / 1000).strftime('%Y')

    # Time range search
    for doc in my_new_list:
        if (int(doc["timestamp_ms_ts_year"]) in range(start_year_para, end_year_para)):
            my_new_list_with_timerange.append(doc)

    return my_new_list_with_timerange


def insert_data():
    with open("./corona-out-2", "r") as f1:
        for line in f1:
            try:
                data = json.loads(line)

                if (data['text'].startswith('RT')):
                    data_retweet_hashtag = {'text': data['text'], 'timestamp_ms': data['timestamp_ms'],
                                            'created_at': data['created_at'], 'name': data['user']['name'],
                                            'Followers_count': data['user']['followers_count'],
                                            'hashtags': data['entities']['hashtags'][0]['text']}
                    collection.insert_one(data_retweet_hashtag)
                else:
                    continue
            except:
                continue

    with open("./corona-out-2", "r") as f1:
        for line in f1:
            try:
                data = json.loads(line)
                if not (data['text'].startswith('RT')):
                    data_tweet_hashtag = {'text': data['text'], 'timestamp_ms': data['timestamp_ms'],
                                          'created_at': data['created_at'], 'name': data['user']['name'],
                                          'Followers_count': data['user']['followers_count'],
                                          'hashtags': data['entities']['hashtags'][0]['text']}
                    collection.insert_one(data_tweet_hashtag)
                else:
                    continue
            except:
                continue


def search_by_hashtag_using_index(name_hashtag_index, start_year_para, end_year_para):
    import datetime
    my_new_list = []
    my_new_list_with_timerange = []
    my_new_list_withtimerange_relevance = []

    # Create text index
    # collection.create_index([('hashtags', 'text')], name='Hashtags_index')

    # Find hashtags with the hashtags that user searched for
    for doc in collection.find({"$text": {"$search": name_hashtag_index}},
                               {"text": 1, "timestamp_ms": 1, "Followers_count": 1}):
        my_new_list.append(doc)

    # Execution statistics for the query
    print(collection.find({"$text": {"$search": name_hashtag_index}},
                          {"text": 1, "timestamp_ms": 1, "Followers_count": 1}).explain()["executionStats"])

    for doc in my_new_list:
        doc["timestamp_ms_ts"] = datetime.datetime.fromtimestamp((int(doc["timestamp_ms"])) / 1000).strftime(
            '%Y-%m-%d %H:%M:%S')
        doc["timestamp_ms_ts_year"] = datetime.datetime.fromtimestamp((int(doc["timestamp_ms"])) / 1000).strftime('%Y')

    # Time range search
    for doc in my_new_list:
        if (int(doc["timestamp_ms_ts_year"]) in range(start_year_para, end_year_para)):
            my_new_list_with_timerange.append(doc)

    # Notion of relevance to rank the search results
    my_new_list_with_timerange_relevance = sorted(my_new_list_with_timerange, key=lambda x: x['Followers_count'],
                                                  reverse=True)

    return my_new_list_with_timerange_relevance

In [4]:
class MyCache:
    """"""

    # ----------------------------------------------------------------------
    def __init__(self):
        """Constructor"""
        self.cache = {}
        self.max_cache_size = 10

    # ----------------------------------------------------------------------
    def __contains__(self, key):
        """
        Returns True or False depending on whether or not the key is in the
        cache
        """
        return key in self.cache

    # ----------------------------------------------------------------------
    def update(self, key, value):
        """
        Update the cache dictionary and optionally remove the oldest item
        """
        if key not in self.cache and len(self.cache) >= self.max_cache_size:
            self.remove_oldest()
        self.cache[key] = value
        return (self.cache)

    # ----------------------------------------------------------------------
    def remove_oldest(self):
        """
        Remove the entry that has the oldest accessed date
        """
        oldest_entry = None
        for key in self.cache:
            if oldest_entry is None:
                oldest_entry = key
            elif self.cache[key]['Followers_count'] < self.cache[oldest_entry][
                'Followers_count']:
                oldest_entry = key
        self.cache.pop(oldest_entry)

    # ----------------------------------------------------------------------
    @property
    def size(self):
        """
        Return the size of the cache
        """
        return len(self.cache)


In [ ]:
#Search App
import time
from flask import Flask, render_template, request
import mysql.connector
import pymongo
from cache import MyCache
import random
import datetime

app = Flask(__name__)

cache = MyCache()

@app.route('/')
def hello():
    return render_template('homepage.html')


def search_by_string(db, collection, text, start_year_para, end_year_para):
    import datetime
    new_list = []
    new_list_with_timerange = []
    new_list_withtimerange_relevance = []
    for doc in db.my_tweet_project.find():
        if (text in doc['text']):
            new_list.append(doc)

    for doc in new_list:
        doc["timestamp_ms_ts"] = datetime.datetime.fromtimestamp((int(doc["timestamp_ms"])) / 1000).strftime(
            '%Y-%m-%d %H:%M:%S')
        doc["timestamp_ms_ts_year"] = datetime.datetime.fromtimestamp((int(doc["timestamp_ms"])) / 1000).strftime('%Y')

    # Time range search
    for doc in new_list:
        if (int(doc["timestamp_ms_ts_year"]) in range(start_year_para, end_year_para)):
            new_list_with_timerange.append(doc)

        # Notion of relevance to rank the search results
    new_list_with_timerange_relevance = sorted(new_list_with_timerange, key=lambda x: x['Followers_count'],
                                               reverse=True)
    return new_list_with_timerange_relevance


def search_by_hashtag(db, collection, name, start_year_para, end_year_para):
    import datetime
    my_new_list = []
    my_new_list_with_timerange = []
    for doc in collection.find({'hashtags': name}, {"text": 1, "timestamp_ms": 1}):
        my_new_list.append(doc)

    print("one")
    for doc in my_new_list:
        doc["timestamp_ms_ts"] = datetime.datetime.fromtimestamp((int(doc["timestamp_ms"])) / 1000).strftime(
            '%Y-%m-%d %H:%M:%S')
        doc["timestamp_ms_ts_year"] = datetime.datetime.fromtimestamp((int(doc["timestamp_ms"])) / 1000).strftime('%Y')
    print("two")

    # Time range search
    for doc in my_new_list:
        if (int(doc["timestamp_ms_ts_year"]) in range(start_year_para, end_year_para)):
            my_new_list_with_timerange.append(doc)
    print("Thre",my_new_list_with_timerange)

    return my_new_list_with_timerange


@app.route('/',methods=['POST'])
def search_engine():
    text = request.form.get('myInput')
    choice_selected = request.form.get("choice")
    s_date = request.form.get("year-start")
    e_date = request.form.get("year-end")

    s_date = int(s_date[:4]) if (len(s_date) > 4) else 2000
    e_date = int(e_date[:4]) if (len(e_date) > 4) else 3000

    if choice_selected == "users":
        try:
            mydb = mysql.connector.connect(
              host="localhost",
              user="root",
              password="12345678",
              database="twitter"
            )
            mycursor = mydb.cursor()
            sql = "SELECT distinct name,screen_name,followers_count,friends_count,statuses_count,case when verified=0 then 'No' else 'Yes' end from USER where name like CONCAT(%s,'%') OR screen_name like CONCAT(%s,'%') order by 3 desc"
            val = (text, text,)

            comp_key = (sql,val)
            results = []
            t1 = time.time()
            if cache.__contains__(comp_key):
                results = list(cache.cache[comp_key])
                print("Inside",len(results))
            else:
                mycursor.execute(sql, val)
                results = mycursor.fetchall()
                print("outsode",len(results))
                cache.update(comp_key, tuple(results))
            t2 = time.time()
            executionTime, docsReturned = t2 - t1, len(results)
            print('Execution Time','{:.5f}'.format(executionTime))
            return render_template("homepage.html",data=results, flag=True, ex_time="{:.3f}".format(executionTime))
        except mysql.connector.Error as err:
            print("Something went wrong: {}".format(err))
    elif choice_selected == "hashtag":
        print("Mongo goes here")
        try:
            client = pymongo.MongoClient("mongodb://localhost:27017")
            db = client['twitter']
            print("Database is created !!")
            print(db)

            collection = db["retweets"]

            my_new_list = []
            my_new_list_with_timerange = []
            my_new_list_withtimerange_relevance = []

            # Create text index
            # collection.create_index([('hashtags', 'text')], name='Hashtags_index')

            # Find hashtags with the hashtags that user searched for
            for doc in collection.find({"$text": {"$search": text}},
                                       {"text": 1, "timestamp_ms": 1, "Followers_count": 1}):
                my_new_list.append(doc)

            # Execution statistics for the query show (executionTime, docsReturned) on UI
            stats = collection.find({"$text": {"$search": text}},{"text": 1, "timestamp_ms": 1, "Followers_count": 1}).explain()["executionStats"]
            executionTime, docsReturned = stats['executionTimeMillis'], stats['nReturned']

            for doc in my_new_list:
                doc["timestamp_ms_ts"] = datetime.datetime.fromtimestamp((int(doc["timestamp_ms"])) / 1000).strftime(
                    '%Y-%m-%d %H:%M:%S')
                doc["timestamp_ms_ts_year"] = datetime.datetime.fromtimestamp(
                    (int(doc["timestamp_ms"])) / 1000).strftime('%Y')

            # Time range search
            for doc in my_new_list:
                if (int(doc["timestamp_ms_ts_year"]) in range(s_date, e_date)):
                    my_new_list_with_timerange.append(doc)

            # Notion of relevance to rank the search results
            my_new_list_with_timerange_relevance = sorted(my_new_list_with_timerange,
                                                          key=lambda x: x['Followers_count'],
                                                          reverse=True)

            #result = [[obj['text'], obj['Followers_count'], obj['timestamp_ms_ts']] for obj in my_new_list_with_timerange_relevance]
            result = [obj['text'] for obj in my_new_list_with_timerange_relevance]

            return render_template("homepage.html",data=result[:5], flag=False, tweet_flag=True, tot_tweets=len(result), ex_time="{:.3f}".format(executionTime))
        except pymongo.connector.Error as err:
            print("Something went wrong: {}".format(err))
    elif choice_selected == "string":
        print("Mongo goes here")
        try:
            client = pymongo.MongoClient("mongodb://localhost:27017")
            db = client['twitter']
            collection = db["retweets"]

            new_list = []
            new_list_with_timerange = []
            new_list_withtimerange_relevance = []
            t1 = time.time()
            for doc in db.retweets.find():
                if (text in doc['text']):
                    new_list.append(doc)

            for doc in new_list:
                doc["timestamp_ms_ts"] = datetime.datetime.fromtimestamp((int(doc["timestamp_ms"])) / 1000).strftime(
                    '%Y-%m-%d %H:%M:%S')
                doc["timestamp_ms_ts_year"] = datetime.datetime.fromtimestamp(
                    (int(doc["timestamp_ms"])) / 1000).strftime('%Y')

            # Time range search
            for doc in new_list:
                if (int(doc["timestamp_ms_ts_year"]) in range(s_date, e_date)):
                    new_list_with_timerange.append(doc)

                # Notion of relevance to rank the search results
            new_list_with_timerange_relevance = sorted(new_list_with_timerange, key=lambda x: x['Followers_count'],
                                                       reverse=True)
            print(new_list_with_timerange_relevance)
            # result = [[obj['text'], obj['Followers_count'], obj['timestamp_ms_ts']] for obj in my_new_list_with_timerange_relevance]
            result = [obj['text'] for obj in new_list_with_timerange_relevance]

            return render_template("homepage.html", data=result[:5], flag=False, tweet_flag=True,
                                   tot_tweets=len(result), ex_time=time.time() - t1)

        except pymongo.connector.Error as err:
            print("Something went wrong: {}".format(err))
    elif choice_selected is None:
        return render_template("homepage.html",data=["Empty"], flag=False, tweet_flag=False)


if __name__ == '__main__':
  app.run()


